# Keras Tuner

This notebook aims to use the Keras Tuner to find the best set of hyperparameters to train a neural network model.


In [ ]:
!pip install keras_tuner

# Importing Libraries

In [ ]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import tempfile
import tensorflow as tf
import urllib.request

from keras_tuner import RandomSearch
from tensorflow import keras
from tensorflow.keras import layers, losses, metrics, optimizers
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

##  Data Preparation

In [ ]:
DATA_DIRECTORY = tempfile.mkdtemp(prefix="keras-tuner-data")
DATA_FILEPATH = os.path.join(DATA_DIRECTORY, "penguins.csv")

urllib.request.urlretrieve(
    "https://storage.googleapis.com/download.tensorflow.org/data/palmer_penguins/penguins_size.csv", 
    DATA_FILEPATH
)

df = pd.read_csv(DATA_FILEPATH)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   culmen_length_mm   342 non-null    float64
 3   culmen_depth_mm    342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                334 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB


# Pipeline

In [ ]:
numerical_columns = [column for column in df.columns if df[column].dtype in ["int64", "float64"]]

numerical_preprocessor = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

categorical_preprocessor = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessor = ColumnTransformer(
    transformers=[
        ("numerical", numerical_preprocessor, numerical_columns),
        ("categorical", categorical_preprocessor, ["island"]),
    ]
)

# Split and Transform

In [ ]:
y = df.species
X = df.drop(["species", "sex"], axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X, 
    y, 
    test_size=0.20, 
    random_state=42
)

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

label_encoder = LabelEncoder()

y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

## Model

A function that builds our model (simple dense network).

There are three different hyperparameters that we want to tune:

* The number of units of the first dense layer. We want to try `4`, `8`, and `12`.
* The number of units of the second dense layer. We are also trying `4`, `8`, and `12`.
* The learning rate. Here we want to try `1e-2` and `1e-3`.

Notice how we are instrumenting our model with a couple of `hp.Int()` an `hp.Choice()` function. This is how the Tuner knows which parameters to tune.



In [ ]:
def _model(hp):
    model = keras.Sequential([
        layers.Dense(
            hp.Int("dense_1_units", min_value=4, max_value=12, step=4, default=8),
            input_shape=(X_train.shape[1],)
            
        ),
        layers.Dense(
            hp.Int("dense_2_units", min_value=4, max_value=12, step=4, default=8), 
            activation="relu"
        ),
        layers.Dense(3, activation="softmax"),
    ])

    model.compile(
        optimizer=optimizers.Adam(
            hp.Choice("learning_rate", values=[1e-2, 1e-3])
        ),
        loss="categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

# Hyperparameter Tuning

Instantiating the tuner using the RandomSearch strategy.

The way the tunner will determine the best model is by looking at the validation accuracy and we are going to be running 10 trials.

Then, we can kick off the search. The signature of the `search()` function is the same as the `model.fit()` function in Keras.

Finally, we can print the results of the tuning process. They will be sorted with the best hyperparameter values at the top.

In [ ]:
tuner = RandomSearch(
    _model,
    objective="val_accuracy",
    max_trials=10,
    overwrite=True,
    directory="keras-tuner",
    project_name="keras-tuner-example",
)

tuner.search_space_summary()

tuner.search(
    X_train[:,:], 
    to_categorical(y_train), 
    epochs=5, 
    validation_data=(X_test[:,:], to_categorical(y_test))
)

tuner.results_summary()

Trial 10 Complete [00h 00m 02s]
val_accuracy: 0.4202898442745209

Best val_accuracy So Far: 0.9855072498321533
Total elapsed time: 00h 00m 29s
Results summary
Results in keras-tuner/keras-tuner-example
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 02 summary
Hyperparameters:
dense_1_units: 8
dense_2_units: 12
learning_rate: 0.01
Score: 0.9855072498321533

Trial 06 summary
Hyperparameters:
dense_1_units: 8
dense_2_units: 4
learning_rate: 0.01
Score: 0.9855072498321533

Trial 08 summary
Hyperparameters:
dense_1_units: 12
dense_2_units: 4
learning_rate: 0.01
Score: 0.9855072498321533

Trial 05 summary
Hyperparameters:
dense_1_units: 4
dense_2_units: 8
learning_rate: 0.01
Score: 0.9710144996643066

Trial 00 summary
Hyperparameters:
dense_1_units: 4
dense_2_units: 12
learning_rate: 0.01
Score: 0.8695651888847351

Trial 04 summary
Hyperparameters:
dense_1_units: 4
dense_2_units: 4
learning_rate: 0.01
Score: 0.7536231875419617

Trial 03 summary
Hyperparameters:

# Predictions

We use the best model that the tuner found to make predictions on the test set.


In [ ]:
best_model = tuner.get_best_models(num_models=1)[0]
y_pred = np.argmax(best_model.predict(X_test), axis=-1)
accuracy = np.sum(y_pred == y_test) / len(y_test) * 100
print(f"Accuracy: {accuracy:.2f}")

3/3 [==============================] - 0s 3ms/step
Accuracy: 98.55
